# Load annotated data

In [53]:
import json
import pprint 
pp = pprint.PrettyPrinter(indent=4)
import numpy as np
import pandas as pd

# dataset_name = 'meme_retrieval_data'
# dataset_name = 'figmemes'
# dataset_name = 'figmemes_cot'
dataset_name = 'figmemes_cot_few_shot'
# dataset_name = 'memecap'

img_prompt_respond_file = ''
gt_data_file = ''
if dataset_name == 'meme_retrieval_data':
    img_prompt_respond_file = '../data/meme_retrieval_data/filtered_meme_configs_5_attributes_meme_retri.json'
elif dataset_name == 'figmemes':
    img_prompt_respond_file = '../data/figmemes/filtered_meme_configs_templatic_without_about_section.json'
    gt_data_file = '../data/figmemes/figmemes_annotations.tsv'
elif dataset_name == 'figmemes_cot':
    img_prompt_respond_file = '../data/figmemes/filtered_meme_configs_figmemes_cot.json'
    gt_data_file = '../data/figmemes/cot_samples_gt.tsv'
elif dataset_name == 'figmemes_cot_few_shot':
    img_prompt_respond_file = '../data/figmemes/filtered_meme_configs_1-11_with_examples_of_labels.json'
    gt_data_file = '../data/figmemes/cot_samples_gt.tsv'
elif  dataset_name == 'memecap':
    img_prompt_respond_file = '../data/memecap/filtered_meme_configs_5_attributes_memecap.json'

# load predicted data
meme_configs = []
with open(img_prompt_respond_file, 'r', encoding='utf-8') as json_file:
    meme_configs = json.load(json_file)
len(meme_configs)

# load ground truth data
gt_data = pd.read_csv(gt_data_file, sep='\t', index_col='img_name')

In [54]:
gt_data

,allusion,exaggeration,irony,anthrop,metaphor,contrast,year,arts,real,mixed,infograph
img_name,,,,,,,,,,,
1458964859691.jpg,1,0,0,0,0,0,2017,1,0,0,0
1464007930878.jpg,0,0,0,0,0,0,2020,0,1,0,0
1467472694126.png,0,0,1,0,0,0,2017,1,0,0,0
1468935750176.jpg,1,1,0,0,0,0,2019,1,0,0,0
1470534047335.jpg,0,0,0,0,0,0,2017,0,1,0,0
1470839669535.jpg,0,0,1,0,0,0,2017,1,0,0,0
1472045568054.jpg,0,0,0,0,0,0,2018,0,1,0,0
1472180356237.jpg,1,1,0,0,1,1,2017,0,0,1,0
1474915955788.png,0,0,0,0,0,1,2017,0,0,1,0


In [55]:
# gt_data.loc['01144951214.png'][:6].tolist()

In [56]:
def label2vec(label_list: list):
    labels = ['allusion', 'exaggeration', 'irony', 'anthrop', 'metaphor', 'contrast']
    vec = np.zeros((1, len(labels)))
    for label in label_list:
        vec[0, labels.index(label)] = 1
    return vec

label2vec(['exaggeration'])

array([[0., 1., 0., 0., 0., 0.]])

In [57]:
meme_configs[0]

{'image_dir': './data/figmemes/images/1458964859691.jpg',
 'prompt': '\nTask: \nYou are going to analyse the literary devices of a meme, and choose suitable literary devices from the given candidates;\nYou need to compare the given meme to these examples.\nLiterary devices: \n**Allusion**: Referencing historical events, figures, symbols, art, literature or pop culture.\n \nExample of allusion: This meme features an edited image of Donald Trump, portraying him in a dramatic pose holding two guns, similar to a movie poster. The text reads "TRUMP: He is come to clean America," suggesting a tough, action-hero persona. The "COMING SOON 2016" refers to the year Donald Trump ran for and won the U.S. presidential election.\n\n**Exaggeration**: Similar to Hyperbole. Use of exaggerated terms for emphasis, including exaggerated visuals (including unrealistic features portraying minorities).\n \nExample of exaggeration:\nThe meme humorously portrays Madagascar\'s extreme response with exaggerated 

In [58]:
predictions = []
gt = []
for idx, meme_conf in enumerate(meme_configs):
    print(idx, meme_conf)
    predictions.append(label2vec(meme_conf['literary device']))
    gt.append(gt_data.loc[meme_conf['image_dir'].split('/')[-1]][:6].tolist())

predictions = np.array(predictions).squeeze()
gt = np.array(gt)
predictions.shape, gt.shape

0 {'image_dir': './data/figmemes/images/1458964859691.jpg', 'prompt': '\nTask: \nYou are going to analyse the literary devices of a meme, and choose suitable literary devices from the given candidates;\nYou need to compare the given meme to these examples.\nLiterary devices: \n**Allusion**: Referencing historical events, figures, symbols, art, literature or pop culture.\n \nExample of allusion: This meme features an edited image of Donald Trump, portraying him in a dramatic pose holding two guns, similar to a movie poster. The text reads "TRUMP: He is come to clean America," suggesting a tough, action-hero persona. The "COMING SOON 2016" refers to the year Donald Trump ran for and won the U.S. presidential election.\n\n**Exaggeration**: Similar to Hyperbole. Use of exaggerated terms for emphasis, including exaggerated visuals (including unrealistic features portraying minorities).\n \nExample of exaggeration:\nThe meme humorously portrays Madagascar\'s extreme response with exaggerated

((11, 6), (11, 6))

In [59]:
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

print('macro:', f1_score(gt.flatten(), predictions.flatten(), average='macro'))
print('micro:', f1_score(gt.flatten(), predictions.flatten(), average='micro'))
print(classification_report(gt.flatten(), predictions.flatten()))

macro: 0.5416666666666667
micro: 0.6363636363636364
              precision    recall  f1-score   support

           0       0.82      0.69      0.75        52
           1       0.27      0.43      0.33        14

    accuracy                           0.64        66
   macro avg       0.55      0.56      0.54        66
weighted avg       0.70      0.64      0.66        66



In [60]:
def gimme_f1s(y_true, y_pred):
    print('zero')
    print(classification_report(y_true=y_true, y_pred=y_pred, zero_division=0))
    print('one')
    print(classification_report(y_true=y_true, y_pred=y_pred, zero_division=1))
    print()
    print('zero')
    f1s = ['micro', 'macro', 'weighted', 'samples']
    for score in f1s:
        f1 = f1_score(y_true=y_true, y_pred=y_pred, zero_division=0, average=score)*100
        print(score)
        print(f1)
        print()
    print('one')
    for score in f1s:
        f1 = f1_score(y_true=y_true, y_pred=y_pred, zero_division=1, average=score)*100
        print(score)
        print(f1)
        print()

gimme_f1s(gt, predictions)

zero
              precision    recall  f1-score   support

           0       0.38      0.75      0.50         4
           1       0.00      0.00      0.00         3
           2       0.33      1.00      0.50         2
           3       0.33      1.00      0.50         1
           4       0.00      0.00      0.00         2
           5       0.00      0.00      0.00         2

   micro avg       0.27      0.43      0.33        14
   macro avg       0.17      0.46      0.25        14
weighted avg       0.18      0.43      0.25        14
 samples avg       0.32      0.36      0.32        14

one
              precision    recall  f1-score   support

           0       0.38      0.75      0.50         4
           1       0.00      0.00      0.00         3
           2       0.33      1.00      0.50         2
           3       0.33      1.00      0.50         1
           4       1.00      0.00      0.00         2
           5       0.00      0.00      0.00         2

   micro avg  